In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import gc
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import activations
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


In [3]:
data = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
data = data.drop(columns=['time_id', 'row_id', 'investment_id', 'target']).values
scaler = StandardScaler()
scaler.fit(data)
del data
gc.collect()

0

In [4]:
class Mish(layers.Layer):
    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)

    def call(self, inputs, *args, **kwargs):
        return inputs * K.tanh(K.softplus(inputs))

In [5]:
input_dim = 300
def build_encoder(input_dim, noise_rate=0.05):
    ori_input = Input(shape=(input_dim,), dtype='float32')
    encode = layers.BatchNormalization()(ori_input)
    encode = layers.GaussianNoise(stddev=noise_rate)(encode)
    encode = layers.Dense(256)(encode)
    encode = Mish()(encode)
    decode = layers.Dropout(0.2)(encode)
    decode = layers.Dense(input_dim, name='decoded')(decode)
    x = layers.Dense(265, use_bias=False)(decode)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1, name='target_out')(x)

    encoder = models.Model(inputs=[ori_input], outputs=[encode])
    autoencoder = models.Model(inputs=[ori_input], outputs=[decode, x])
    return autoencoder, encoder

In [6]:
def build_encoder_0(input_dim, noise_rate=0.05):
    ori_input = Input(shape=(input_dim,), dtype='float32')
    encode = layers.BatchNormalization()(ori_input)
    encode = layers.GaussianNoise(stddev=noise_rate)(encode)
    encode = layers.Dense(256)(encode)
    encode = Mish()(encode)
    decode = layers.Dropout(0.1)(encode)
    decode = layers.Dense(input_dim, name='decoded')(decode)
    x = layers.Dense(265, use_bias=False)(decode)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(1, name='target')(x)

    encoder = models.Model(inputs=[ori_input], outputs=[encode])
    autoencoder = models.Model(inputs=[ori_input], outputs=[decode, x])
    return autoencoder, encoder

In [7]:
def build_model_0(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [8]:
def build_model_1(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [9]:
def build_model_2(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])

    return model

In [10]:
def build_model_bigdrop_1(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [11]:
def build_model_bigdrop(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [12]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()  

In [13]:
weights = ['../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-1-ep8-loss0.8170.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-10-ep8-loss0.8171.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-11-ep8-loss0.8174.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-12-ep8-loss0.8181.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-2-ep8-loss0.8170.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-3-ep8-loss0.8181.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-4-ep8-loss0.8191.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-5-ep8-loss0.8176.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-6-ep8-loss0.8170.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-7-ep8-loss0.8175.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-9-ep8-loss0.8182.h5',
          
           '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-1-ep8-loss0.8173.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-10-ep8-loss0.8175.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-11-ep8-loss0.8181.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-12-ep8-loss0.8185.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-2-ep8-loss0.8176.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-3-ep8-loss0.8182.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-4-ep8-loss0.8178.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-5-ep8-loss0.8183.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-6-ep8-loss0.8185.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-7-ep8-loss0.8171.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-8-ep8-loss0.8176.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-9-ep8-loss0.8172.h5']

model_bigdrop_bt512 = []
for w in weights:
    autoencoder, encoder = build_encoder_0(input_dim)
    encoder.trainable = False
    model_temp = build_model_bigdrop(encoder)
    model_temp.load_weights(w)
    model_bigdrop_bt512.append(model_temp)
print(len(model_bigdrop_bt512))

2022-04-19 08:19:27.861059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 08:19:27.971042: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 08:19:27.971819: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 08:19:27.973111: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

23


In [14]:
weights = ['../input/ubiquantbigdropbt1024weights/fulltr-bigdrop-origin-bt1024-test-1-ep9-loss0.8211.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-10-ep9-loss0.8189.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-2-ep9-loss0.8203.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-3-ep9-loss0.8210.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-4-ep9-loss0.8219.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-5-ep9-loss0.8202.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-6-ep9-loss0.8196.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-7-ep9-loss0.8201.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-8-ep9-loss0.8198.h5',
          '../input/ubiquantbigdropbt1024weights/origin-bigdrop-bt1024-test-9-ep9-loss0.8196.h5',
           
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-1-ep9-loss0.8200.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-10-ep9-loss0.8206.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-11-ep9-loss0.8205.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-12-ep9-loss0.8193.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-13-ep9-loss0.8211.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-14-ep9-loss0.8209.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-2-ep9-loss0.8217.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-3-ep9-loss0.8200.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-4-ep9-loss0.8191.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-5-ep9-loss0.8198.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-6-ep9-loss0.8203.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-7-ep9-loss0.8195.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-8-ep9-loss0.8198.h5',
          '../input/ubiquantbigdrop1bt1024weights/origin-bigdrop-1-bt1024-test-9-ep9-loss0.8200.h5']

model_bigdrop_bt1024 = []
for w in weights:
    autoencoder, encoder = build_encoder_0(input_dim)
    encoder.trainable = False
    model_temp = build_model_bigdrop(encoder)
    model_temp.load_weights(w)
    model_bigdrop_bt1024.append(model_temp)
print(len(model_bigdrop_bt1024))

24


In [15]:
weights = ['../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-1-ep9-loss0.8195.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-10-ep9-loss0.8197.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-11-ep9-loss0.8204.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-12-ep9-loss0.8198.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-13-ep9-loss0.8210.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-14-ep9-loss0.8192.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-15-ep9-loss0.8188.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-16-ep9-loss0.8202.h5',
#           '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-17-ep9-loss0.8217.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-18-ep9-loss0.8198.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-19-ep9-loss0.8206.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-2-ep9-loss0.8202.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-20-ep9-loss0.8193.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-21-ep9-loss0.8201.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-22-ep9-loss0.8200.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-23-ep9-loss0.8195.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-24-ep9-loss0.8213.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-25-ep9-loss0.8190.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-3-ep9-loss0.8199.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-4-ep9-loss0.8200.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-5-ep9-loss0.8212.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-6-ep9-loss0.8199.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-7-ep9-loss0.8206.h5',
          '../input/ubiquantbigdrop-new-bt1024weights/origin-bigdrop-1-bt1024-test-9-ep9-loss0.8190.h5']

model_bigdrop_1_bt1024 = []
for w in weights:
    autoencoder, encoder = build_encoder_0(input_dim)
    encoder.trainable = False
    model_temp = build_model_bigdrop_1(encoder)
    model_temp.load_weights(w)
    model_bigdrop_1_bt1024.append(model_temp)
print(len(model_bigdrop_1_bt1024))

23


In [16]:
weights = ['../input/ubiquanttts6foldweights/origin-bigdrop-test-1_5.h5',
          '../input/ubiquanttts6foldweights/origin-bigdrop-test-2_5.h5',
          '../input/ubiquanttts6foldweights/origin-bigdrop-test-3_5.h5',
          '../input/ubiquanttts6foldweights/origin-bigdrop-test-4_5.h5']

model_bigdrop_fold6 = []

autoencoder, encoder = build_encoder_0(input_dim)
encoder.trainable = False
model_temp = build_model_bigdrop_1(encoder)
model_temp.load_weights('../input/ubiquanttts6foldweights/origin-bigdrop-1-test-1_5.h5')
model_bigdrop_fold6.append(model_temp)

for w in weights:
    autoencoder, encoder = build_encoder_0(input_dim)
    encoder.trainable = False
    model_temp = build_model_bigdrop(encoder)
    model_temp.load_weights(w)
    model_bigdrop_fold6.append(model_temp)
print(len(model_bigdrop_fold6))

5


In [17]:
weights = ['../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-1_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-2_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-3_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-4_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-5_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-6_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-7_13.h5',
          '../input/ubiquantbigdrop14foldweights/origin-bigdrop-1-14fold-test-8_13.h5']

model_bigdrop_fold14 = []
for w in weights:
    autoencoder, encoder = build_encoder_0(input_dim)
    encoder.trainable = False
    model_temp = build_model_bigdrop(encoder)
    model_temp.load_weights(w)
    model_bigdrop_fold14.append(model_temp)
print(len(model_bigdrop_fold14))

8


In [18]:
autoencoder, encoder_0 = build_encoder(input_dim)
encoder_0.trainable = False
autoencoder, encoder_1 = build_encoder(input_dim)
encoder_1.trainable = False
autoencoder, encoder_2 = build_encoder(input_dim)
encoder_2.trainable = False

autoencoder, encoder_0_0 = build_encoder(input_dim)
encoder_0_0.trainable = False
autoencoder, encoder_0_2 = build_encoder(input_dim)
encoder_0_2.trainable = False

autoencoder, encoder_1_0 = build_encoder(input_dim)
encoder_1_0.trainable = False

autoencoder, encoder_2_0 = build_encoder(input_dim)
encoder_2_0.trainable = False
autoencoder, encoder_2_1 = build_encoder(input_dim)
encoder_2_1.trainable = False
autoencoder, encoder_2_2 = build_encoder(input_dim)
encoder_2_2.trainable = False
autoencoder, encoder_2_3 = build_encoder(input_dim)
encoder_2_3.trainable = False
autoencoder, encoder_2_4 = build_encoder(input_dim)
encoder_2_4.trainable = False

In [19]:
#0.1521
model_0 = build_model_0(encoder_0)
model_0.load_weights('../input/ubiquantfulltrainsubweights/m0-fulltr-two-256-drop0.2-encodrop0.2-ep7-loss0.8171.h5')
#0.1515
model_0_0= build_model_0(encoder_0_0)
model_0_0.load_weights('../input/ubiquantfulltrainsubweights/m0_0-0.1515-fulltr-two256drop-seed-1-test-3-ep7-loss0.8170.h5')
#1532
model_0_2= build_model_0(encoder_0_2)
model_0_2.load_weights('../input/ubiquantfulltrainsubweights/m0_2_fulltr-two256drop-myloss-4-6-test-1-ep7-loss0.8179.h5')


#1521-ep7
model_1 = build_model_1(encoder_1)
model_1.load_weights('../input/ubiquantfulltrainsubweights/m1-fulltr-256dropmove-encodrop0.2-ep7-loss0.8178.h5')
#1520
model_1_0 = build_model_1(encoder_1_0)
model_1_0.load_weights('../input/ubiquantfulltrainsubweights/m1_0_fulltr-dropmove-myloss-4-6-test-1-ep7-loss0.8170.h5')

#1523
model_2 = build_model_2(encoder_2)
model_2.load_weights('../input/ubiquantfulltrainsubweights/m2-fulltr-Mish-512_128-512-newenco-ep7-loss0.8178.h5')
#1501
model_2_0 = build_model_2(encoder_2_0)
model_2_0.load_weights('../input/ubiquantfulltrainsubweights/m2_0_fulltr-origin-seed-1-test-3-ep7-loss0.8171.h5')
#1514
model_2_1 = build_model_2(encoder_2_1)
model_2_1.load_weights('../input/ubiquantfulltrainsubweights/m2_1_fulltr-tfrecords-original-test-1-ep7-loss0.8189.h5')
#1511
model_2_2 = build_model_2(encoder_2_2)
model_2_2.load_weights('../input/ubiquantfulltrainsubweights/m2_2_fulltr-origin-seed430-test-1-ep7-loss0.8179.h5')
#1504
model_2_3 = build_model_2(encoder_2_3)
model_2_3.load_weights('../input/ubiquantfulltrainsubweights/m2_3_fulltr-origin-seed72-test-1-ep7-loss0.8176.h5')
#1516
model_2_4 = build_model_2(encoder_2_4)
model_2_4.load_weights('../input/ubiquantfulltrainsubweights/m2_4_fulltr-tfrecords-origin-test-3-ep7-loss0.8170.h5')



In [20]:
def make_test_dataset(feature, batch_size=2048):
    ds = tf.data.Dataset.from_tensor_slices(((feature)))
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

In [21]:
for (test_df, sample_prediction_df) in iter_test:
    try:
        data_ox = test_df.drop(columns=['row_id', 'investment_id']).values
        data_ox = scaler.transform(data_ox)
        data_ox = make_test_dataset(data_ox)

        t0 = (model_0.predict(data_ox) * 1.521 +
              model_0_0.predict(data_ox) * 1.515 +
             model_0_2.predict(data_ox) * 1.532 +
             model_1.predict(data_ox) * 1.521 +
             model_1_0.predict(data_ox) * 1.520 +
             model_2.predict(data_ox) * 1.523 +
             model_2_0.predict(data_ox) * 1.501 +
             model_2_1.predict(data_ox) * 1.514 +
             model_2_2.predict(data_ox) * 1.511 +
             model_2_4.predict(data_ox) * 1.516) 
        t0[np.isnan(t0)] = 0
        t0 = t0 / 15.0
        
        t1 = model_bigdrop_bt512[0].predict(data_ox)
        for i in range(1, len(model_bigdrop_bt512)):
            t1 = t1 + model_bigdrop_bt512[i].predict(data_ox)
        t1[np.isnan(t1)] = 0
        t1 = t1 / len(model_bigdrop_bt512)

        t2 = model_bigdrop_bt1024[0].predict(data_ox)
        for i in range(1, len(model_bigdrop_bt1024)):
            t2 = t2 + model_bigdrop_bt1024[i].predict(data_ox)
        t2[np.isnan(t2)] = 0
        t2 = t2 / len(model_bigdrop_bt1024)

        t3 = model_bigdrop_1_bt1024[0].predict(data_ox)
        for i in range(1, len(model_bigdrop_1_bt1024)):
            t3 = t3 + model_bigdrop_1_bt1024[i].predict(data_ox)
        t3[np.isnan(t3)] = 0
        t3 = t3 / len(model_bigdrop_1_bt1024)
        
        t4 = model_bigdrop_fold6[0].predict(data_ox)
        for i in range(1, len(model_bigdrop_fold6)):
            t4 = t4 + model_bigdrop_fold6[i].predict(data_ox)
        t4[np.isnan(t4)] = 0
        t4 = t4 / len(model_bigdrop_fold6)
        
        t5 = model_bigdrop_fold14[0].predict(data_ox)
        for i in range(1, len(model_bigdrop_fold14)):
            t5 = t5 + model_bigdrop_fold14[i].predict(data_ox)
        t5[np.isnan(t5)] = 0
        t5 = t5 / len(model_bigdrop_fold14)


        sample_prediction_df['target'] = (t0 + t1 + t2 + t3) / 4.0 * 0.65 + t4 * 0.125 + t5 * 0.225 

    except Exception:
        sample_prediction_df['target'] = np.random.randn(len(sample_prediction_df))
    
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


2022-04-19 08:19:57.857780: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
